In [ ]:
from pandas import DataFrame, read_csv
from collections import defaultdict
import pandas as pd
import numpy as np
import math

trees_csv = 'data/ME_TREE.csv'
trees_df = pd.read_csv(trees_csv,usecols=['INVYR','PLOT','SUBP','SPGRPCD','DIA','DIACALC','PREVDIA'])

In [ ]:
"""
plots: {
    <plot_num>: {
        <year>: {
            <species_code>: {
                'trees_num': <num>,
                'basal_area': <ba>,
                'frequency': <freq>,
                'importance_value': <iv>
             }
         }
     }
}

plot_subp: {
    <plot_num>: {
        <year>: {
            <species_code>: {
                set(subplots)
            }
        }
    }
 }

"""
#The set stores in which subplots does the species appear
#Both plots and plots_subp have 'TOTAL' as a species code.  It stores the sum of all the species
#This step takes a while (96 seconds in Athena computer)
plots = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:defaultdict(float))))
plots_subp = defaultdict(lambda:defaultdict(lambda:defaultdict(set)))
for i, row in trees_df.iterrows():
    plots[row['PLOT']][row['INVYR']]['TOTAL']['num'] += 1
    plots[row['PLOT']][row['INVYR']][row['SPGRPCD']]['num'] += 1
    plots_subp[row['PLOT']][row['INVYR']][row['SPGRPCD']].add(row['SUBP'])
    plots_subp[row['PLOT']][row['INVYR']]['TOTAL'].add(row['SUBP'])
    
    if not np.isnan(row['DIA']):
        dia = row['DIA']
    else:
        dia = row['DIACALC'] if not np.isnan(row['DIACALC']) else row['PREVDIA']
        
    plots[row['PLOT']][row['INVYR']]['TOTAL']['ba'] += math.pi*(dia**2)/4
    plots[row['PLOT']][row['INVYR']][row['SPGRPCD']]['ba'] += math.pi*(dia**2)/4

In [ ]:
for plot, a1 in plots_subp.iteritems():
    for year, a2 in plots_subp[plot].iteritems():
        for species, count in plots_subp[plot][year].iteritems():
            plots[plot][year][species]['freq'] = len(plots_subp[plot][year][species])/float(len(plots_subp[plot][year]['TOTAL']))
            plots[plot][year]['TOTAL']['freq'] += len(plots_subp[plot][year][species])/float(len(plots_subp[plot][year]['TOTAL']))

In [ ]:
for plot, a1 in plots.iteritems():
    for year, a2 in plots[plot].iteritems():
        for species, a3 in plots[plot][year].iteritems():
            plots[plot][year][species]['iv'] = (plots[plot][year][species]['freq']/(3.0*plots[plot][year]['TOTAL']['freq']) 
                                             + plots[plot][year][species]['ba']/(3.0*plots[plot][year]['TOTAL']['ba']) 
                                             + plots[plot][year][species]['num']/(3.0*plots[plot][year]['TOTAL']['num']))

In [ ]:
import json
with open('data/points.json', 'w') as outfile:
    json.dump(plots,outfile)

In [ ]:
remap=dict()
cur=4
num_plots=0
for plot, a1 in plots.iteritems():
    for year, a2 in plots[plot].iteritems():
        num_plots += 1
        for species, a3 in plots[plot][year].iteritems():
            if species not in remap and species!='TOTAL':
                remap[species]=cur
                cur+=1
data_points = pd.DataFrame(0.0, index=np.arange(num_plots), columns=np.arange(cur))
cur=0
for plot, a1 in plots.iteritems():
    for year, a2 in plots[plot].iteritems():
        data_points.iloc[cur][0] = plot
        data_points.iloc[cur][1] = year
        data_points.iloc[cur][2] = plots[plot][year]['TOTAL']['num']
        data_points.iloc[cur][3] = plots[plot][year]['TOTAL']['ba']
        for species, a3 in plots[plot][year].iteritems():
            if (species!='TOTAL'): data_points.iloc[cur][remap[species]] = plots[plot][year][species]['iv']
        cur+=1
data_points.to_csv('data/points.csv',index=False,header=False)